<a href="https://colab.research.google.com/github/abdulla41mamun/CSE710-Codes/blob/main/BaumWelch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np

# Initialization of HMM parameters
pi = np.array([0.6, 0.4])  # Initial state probabilities [H, S]
A = np.array([[0.8, 0.2], [0.8, 0.2]])  # Transition probabilities
phi = np.array([[0.5, 0.2, 0.2, 0.1], [0.1, 0.3, 0.2, 0.4]])  # Emission probabilities for PP, PN, NP, NN

# Observation sequence: PP, PP, NP, NN
obs = [0, 0, 2, 3]  # Encoded as indices
T = len(obs)
num_states = len(pi)

# Forward Algorithm
alpha = np.zeros((T, num_states))
alpha[0, :] = pi * phi[:, obs[0]]

for t in range(1, T):
    for j in range(num_states):
        alpha[t, j] = np.sum(alpha[t - 1, :] * A[:, j]) * phi[j, obs[t]]

# Backward Algorithm
beta = np.zeros((T, num_states))
beta[T - 1, :] = 1

for t in range(T - 2, -1, -1):
    for i in range(num_states):
        beta[t, i] = np.sum(A[i, :] * phi[:, obs[t + 1]] * beta[t + 1, :])

# Compute gamma (state probabilities)
gamma = np.zeros((T, num_states))
for t in range(T):
    gamma[t, :] = alpha[t, :] * beta[t, :]
    gamma[t, :] /= np.sum(gamma[t, :])

# Compute xi (transition probabilities) - CORRECTED NORMALIZATION
xi = np.zeros((T - 1, num_states, num_states))
for t in range(T - 1):
    for i in range(num_states):
        for j in range(num_states):
            xi[t, i, j] = (
                alpha[t, i] * A[i, j] * phi[j, obs[t + 1]] * beta[t + 1, j]
            )

# Normalize xi AFTER summing over all i, j, and t
xi /= np.sum(xi)  # Correct normalization

# Update Parameters
updated_pi = gamma[0, :]
updated_A = np.zeros((num_states, num_states))
for i in range(num_states):
    for j in range(num_states):
        updated_A[i, j] = np.sum(xi[:, i, j]) / np.sum(gamma[:-1, i])

updated_phi = np.zeros_like(phi)
for j in range(num_states):
    for k in range(phi.shape[1]):
        mask = (np.array(obs) == k)
        updated_phi[j, k] = np.sum(gamma[mask, j]) / np.sum(gamma[:, j])

# Print Results
print("Forward Probabilities (alpha):")
print(alpha)
print("\nBackward Probabilities (beta):")
print(beta)
print("\nState Probabilities (gamma):")
print(gamma)
print("\nTransition Probabilities (xi):")
print(xi)
print("\nUpdated Initial Probabilities (pi):")
print(updated_pi)
print("\nUpdated Transition Probabilities (A):")
print(updated_A)
print("\nUpdated Emission Probabilities (phi):")
print(updated_phi)

Forward Probabilities (alpha):
[[0.3       0.04     ]
 [0.136     0.0068   ]
 [0.022848  0.005712 ]
 [0.0022848 0.0022848]]

Backward Probabilities (beta):
[[0.01344 0.01344]
 [0.032   0.032  ]
 [0.16    0.16   ]
 [1.      1.     ]]

State Probabilities (gamma):
[[0.88235294 0.11764706]
 [0.95238095 0.04761905]
 [0.8        0.2       ]
 [0.5        0.5       ]]

Transition Probabilities (xi):
[[[0.28011204 0.0140056 ]
  [0.03734827 0.00186741]]

 [[0.25396825 0.06349206]
  [0.01269841 0.0031746 ]]

 [[0.13333333 0.13333333]
  [0.03333333 0.03333333]]]

Updated Initial Probabilities (pi):
[0.88235294 0.11764706]

Updated Transition Probabilities (A):
[[0.25331349 0.08001985]
 [0.22827198 0.10506135]]

Updated Emission Probabilities (phi):
[[0.58529175 0.         0.25520508 0.15950317]
 [0.19100032 0.         0.23114276 0.57785691]]
